In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
books = pd.read_csv('/content/drive/My Drive/Recommender System End-to-End/books_dataset.csv')
ratings = pd.read_csv('/content/drive/My Drive/Recommender System End-to-End/ratings_dataset.csv')
users = pd.read_csv('/content/drive/My Drive/Recommender System End-to-End/users_dataset.csv')

In [4]:
books.head()

,isbn,book_title,book_author,year_of_pub,publisher,image_url_small,image_url_med,image_url_large
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
ratings.head()

,user_id,isbn,book_rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
users['age'] = users['age'].fillna(round(users.age.mean())).astype(int)

In [7]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",35
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",35
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",35


In [8]:
df = users.join(ratings.set_index('user_id'), on='user_id')

In [9]:
df = df.join(books.set_index('isbn'), on='isbn')

In [10]:
df.head()

,user_id,location,age,isbn,book_rating,book_title,book_author,year_of_pub,publisher,image_url_small,image_url_med,image_url_large
0,1,"nyc, new york, usa",35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"stockton, california, usa",18,0195153448,0.0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
2,3,"moscow, yukon territory, russia",35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,"porto, v.n.gaia, portugal",17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"farnborough, hants, united kingdom",35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1217888 entries, 0 to 278857
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   user_id          1217888 non-null  int64  
 1   location         1217888 non-null  object 
 2   age              1217888 non-null  int64  
 3   isbn             1031136 non-null  object 
 4   book_rating      1031136 non-null  float64
 5   book_title       1031136 non-null  object 
 6   book_author      1031135 non-null  object 
 7   year_of_pub      1031136 non-null  float64
 8   publisher        1031136 non-null  object 
 9   image_url_small  1031136 non-null  object 
 10  image_url_med    1031136 non-null  object 
 11  image_url_large  1031132 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 120.8+ MB


In [12]:
df = df.iloc[0:100]
len(df)

100

In [13]:
# once we have the data ready, we can proceed with creating a user-item matrix for collaborative filtering

from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import pandas as pd

# fill in the NaN values with 0s to proceed
sparse_matrix = df.fillna(0)

In [14]:
sparse_matrix.head()

,user_id,location,age,isbn,book_rating,book_title,book_author,year_of_pub,publisher,image_url_small,image_url_med,image_url_large
0,1,"nyc, new york, usa",35,0,0.0,0,0,0.0,0,0,0,0
1,2,"stockton, california, usa",18,0195153448,0.0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
2,3,"moscow, yukon territory, russia",35,0,0.0,0,0,0.0,0,0,0,0
3,4,"porto, v.n.gaia, portugal",17,0,0.0,0,0,0.0,0,0,0,0
4,5,"farnborough, hants, united kingdom",35,0,0.0,0,0,0.0,0,0,0,0


In [15]:
sparse_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 53
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          100 non-null    int64  
 1   location         100 non-null    object 
 2   age              100 non-null    int64  
 3   isbn             100 non-null    object 
 4   book_rating      100 non-null    float64
 5   book_title       100 non-null    object 
 6   book_author      100 non-null    object 
 7   year_of_pub      100 non-null    float64
 8   publisher        100 non-null    object 
 9   image_url_small  100 non-null    object 
 10  image_url_med    100 non-null    object 
 11  image_url_large  100 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 10.2+ KB


In [16]:
#converting the rating and year_of_pub to int because they are in float
sparse_matrix['book_rating'] = sparse_matrix['book_rating'].astype(int)
sparse_matrix['year_of_pub'] = sparse_matrix['year_of_pub'].astype(int)

In [17]:
sparse_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 53
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          100 non-null    int64 
 1   location         100 non-null    object
 2   age              100 non-null    int64 
 3   isbn             100 non-null    object
 4   book_rating      100 non-null    int64 
 5   book_title       100 non-null    object
 6   book_author      100 non-null    object
 7   year_of_pub      100 non-null    int64 
 8   publisher        100 non-null    object
 9   image_url_small  100 non-null    object
 10  image_url_med    100 non-null    object
 11  image_url_large  100 non-null    object
dtypes: int64(4), object(8)
memory usage: 10.2+ KB


In [18]:
#sparse_matrix = sparse_matrix.iloc[:100]

In [19]:
# csr matrix can't deal with non-numeric values, so we convert to category type and use cat.codes to get the numeric representations
sparse_matrix['user_id'] = sparse_matrix['user_id'].astype('category').cat.codes
sparse_matrix['isbn'] = sparse_matrix['isbn'].astype('category').cat.codes

In [34]:
# create a user-item matrix containing the user_id, isbn and rating as the target
user_item_matrix = csr_matrix((sparse_matrix['book_rating'], (sparse_matrix['user_id'], sparse_matrix['isbn'])))

# the maximum n_components that can be used in min(rows, columns) in the sparse_matrix
svd = TruncatedSVD(n_components=7, random_state=42)
user_item_matrix_transformed = svd.fit_transform(user_item_matrix)

In [35]:
explained_variance = svd.explained_variance_ratio_.sum()
print(f"Explained variance of the SVD step: {explained_variance}")

Explained variance of the SVD step: 0.6886109282422648


In [27]:
# Creating a user-item rating matrix
user_item_rating_matrix = df.pivot_table(index='user_id', columns='isbn', values='book_rating')
user_item_rating_matrix.head()

isbn,0002005018,0060168013,0060914068,0060973129,0061076031,0061099686,0156047624,0195153448,0245542957,0312978383,...,1558746218,1567407781,1575663937,1841721522,1879384493,1881320189,3404921038,3442353866,3442410665,3442446937
user_id,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,6.0,6.0,NaN,NaN,7.0,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN


In [28]:
from sklearn.metrics.pairwise import cosine_similarity

# Replace NaN values with 0
user_item_rating_matrix_filled = user_item_rating_matrix.fillna(0)

# Compute cosine similarity between books
item_similarity = cosine_similarity(user_item_rating_matrix_filled.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_rating_matrix.columns, columns=user_item_rating_matrix.columns)
item_similarity_df.head()

isbn,0002005018,0060168013,0060914068,0060973129,0061076031,0061099686,0156047624,0195153448,0245542957,0312978383,...,1558746218,1567407781,1575663937,1841721522,1879384493,1881320189,3404921038,3442353866,3442410665,3442446937
isbn,,,,,,,,,,,,,,,,,,,,,
0002005018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0060168013,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060914068,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060973129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0061076031,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
def recommend_books(isbn, similarity_data, items, k=5):
    """
    Recommends books similar to the given book ISBN.

    :param isbn: The ISBN of the book to find recommendations for.
    :param similarity_data: A DataFrame of item similarity.
    :param items: The original DataFrame of items (books).
    :param k: The number of recommendations to return.
    :return: A list of ISBNs of recommended books.
    """
    # Retrieve similar books
    similar_scores = similarity_data[isbn]
    similar_scores = similar_scores.sort_values(ascending=False)

    # Get top k most similar books
    top_items = similar_scores.iloc[1:k+1].index

    recommended_books = items[items['isbn'].isin(top_items)]
    return recommended_books[['isbn', 'book_title', 'book_author']]

# Example usage
recommend_books('0060168013', item_similarity_df, books)

,isbn,book_title,book_author
10,0771074670,Nights Below Station Street,David Adams Richards
11,080652121X,Hitler's Secret Bankers: The Myth of Swiss Neu...,Adam Lebor
29,0684823802,OUT OF THE SILENT PLANET,C.S. Lewis
50,0812523873,Proxies,Laura J. Mixon
51,0842342702,Left Behind: A Novel of the Earth's Last Days ...,Tim Lahaye


In [ ]:
!nvcc --version

In [ ]:
"""
import torch

# Check if CUDA is available and set the default device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create tensors and move them to GPU
x = torch.rand(1000, 1000, device=device)
y = torch.rand(1000, 1000, device=device)

# Perform a computation on the GPU
z = x + y  # This addition runs on GPU
print(z)

#checking if cuda is available
print(device)
"""